In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
init_load_flag = dbutils.widgets.get("init_load_flag")

### **Data Reading**

In [0]:
df = spark.sql("select * from databricks_cata.silver.customers_silver")

In [0]:
df.display()

### **Removing Duplicates**

In [0]:
df = df.dropDuplicates(subset=['customer_id'])

### **Dividing New vs Old Records**

In [0]:
if init_load_flag ==0:

    df_old = spark.sql("select DimCustomerKey, customer_id, create_date, update_date from databricks_cata.gold.DimCustomers")

else:
    
    df_old = spark.sql('''select 0 DimCustomerKey, 0 customer_id, 0 create_date, 0 update_date from databricks_cata.silver.customers_silver where 1=0''')

In [0]:
df_old.display()

### **Renaming columns of df_old**

In [0]:
df_old = df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey")\
    .withColumnRenamed("customer_id", "old_customer_id")\
    .withColumnRenamed("create_date", "old_create_date")\
    .withColumnRenamed("update_date", "old_update_date")

### **Applying Join with Old Records**

In [0]:
df_join = df.join(df_old, df["customer_id"] == df_old["old_customer_id"], 'left')


In [0]:
df_join.display()

### **Surrogate Key - All the Values**

In [0]:
df = df.orderBy("customer_id").withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))

In [0]:
df.display()

### **Separating New vs Old Records**

In [0]:
 df_new = df_join.filter(df_join['old_DimCustomerKey'].isNull())

In [0]:
df_old = df_join.filter(df_join['old_DimCustomerKey'].isNotNull())

### **Preparing df_old**